# Notebook 3: Comparing Image De-Enhancement Methods

In this notebook, we compare two major approaches to "de-enhancing" or simplifying an image:

1. **Pixel Downsampling (Resolution Reduction)**  
   - The real-world way of reducing resolution.  
   - Literally decreases the number of pixels by resizing the image.  
   - Results in a blocky, pixelated look.  

2. **SVD Truncation (Singular Value Dropping)**  
   - A linear algebra method using the Singular Value Decomposition.  
   - Instead of reducing pixel count, we drop some singular values that represent fine details.  
   - Produces a smoother, blurred look instead of blockiness.  

We’ll apply both methods to the same images and discuss why you might use one method or the other.

In [2]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
from skimage import io, transform, color
import sys
import os

# Add src/ to Python path
sys.path.append(os.path.abspath("../src/svd_image_lab"))
from svd_utils import compute_svd, reconstruct_matrix

# Load sample images
images = {
    "Lenna": io.imread("images/lenna.png"),
    "Camera": io.imread("images/camera.png"),
    "Landscape": io.imread("images/landscape.png")
}

# Convert to grayscale for consistency
for name, img in images.items():
    if img.ndim == 3:
        images[name] = color.rgb2gray(img)


ImportError: cannot import name 'compute_svd' from 'svd_utils' (/Users/jonah/Documents/Coding/Personal/GitHub/svd-image-lab/src/svd_image_lab/svd_utils.py)

## Part 1: Pixel Downsampling (Resolution Reduction)

This is the standard way of lowering image resolution.  
We resize the image to a smaller width and height, then scale it back up to its original size for comparison.

- **What’s happening?**  
  Fewer pixels → less detail is captured.  
  When scaled back up, the image looks blocky or pixelated.  

- **Why do this in practice?**  
  - Save storage and memory.  
  - Speed up image processing tasks.  
  - Simulate how an image looks on low-resolution devices.  

In [ ]:
def downsample_image(img, factor):
    """Downsample an image by a given factor and then rescale back up."""
    small = transform.resize(img, (img.shape[0] // factor, img.shape[1] // factor), anti_aliasing=True)
    return transform.resize(small, img.shape, anti_aliasing=False)

factor = 4  # downsample by 4x
fig, axes = plt.subplots(len(images), 2, figsize=(8, 12))

for i, (name, img) in enumerate(images.items()):
    downsampled = downsample_image(img, factor)
    axes[i, 0].imshow(img, cmap="gray")
    axes[i, 0].set_title(f"{name} - Original")
    axes[i, 1].imshow(downsampled, cmap="gray")
    axes[i, 1].set_title(f"{name} - Downsampled (factor {factor})")

    for ax in axes[i]:
        ax.axis("off")

plt.tight_layout()
plt.show()

## Part 2: SVD Truncation (Dropping Singular Values)

This is a mathematical method for reducing detail.  
Instead of changing pixel resolution, we drop smaller singular values from the decomposition.

- **What’s happening?**  
  Large singular values keep overall structure and shapes.  
  Small singular values hold fine details and noise.  
  Dropping them smooths the image.  

- **Why do this in practice?**  
  - Image compression (store fewer numbers).  
  - Denoising (remove small, noisy singular values).  
  - Feature extraction (keep only the strongest patterns).  

In [ ]:
def svd_truncate(img, keep_k):
    """Reconstruct image using only the top k singular values."""
    U, S, Vt = compute_svd(img)
    S_trunc = np.zeros_like(S)
    S_trunc[:keep_k] = S[:keep_k]
    return reconstruct_matrix(U, S_trunc, Vt)

k = 50  # number of singular values to keep
fig, axes = plt.subplots(len(images), 2, figsize=(8, 12))

for i, (name, img) in enumerate(images.items()):
    svd_recon = svd_truncate(img, k)
    axes[i, 0].imshow(img, cmap="gray")
    axes[i, 0].set_title(f"{name} - Original")
    axes[i, 1].imshow(svd_recon, cmap="gray")
    axes[i, 1].set_title(f"{name} - SVD Truncation (k={k})")

    for ax in axes[i]:
        ax.axis("off")

plt.tight_layout()
plt.show()

## Part 3: Side-by-Side Comparison

Now, let’s put both methods together.  
You’ll see that:

- **Downsampling** → blocky, pixelated look.  
- **SVD truncation** → smooth, blurry look.  

Both lose detail, but in different ways.  

In [ ]:
factor = 4
k = 50

fig, axes = plt.subplots(len(images), 3, figsize=(12, 12))

for i, (name, img) in enumerate(images.items()):
    downsampled = downsample_image(img, factor)
    svd_recon = svd_truncate(img, k)

    axes[i, 0].imshow(img, cmap="gray")
    axes[i, 0].set_title(f"{name} - Original")

    axes[i, 1].imshow(downsampled, cmap="gray")
    axes[i, 1].set_title("Downsampled")

    axes[i, 2].imshow(svd_recon, cmap="gray")
    axes[i, 2].set_title(f"SVD Truncated (k={k})")

    for ax in axes[i]:
        ax.axis("off")

plt.tight_layout()
plt.show()


# Final Discussion

### What can we learn?

- **Pixel Downsampling** reduces resolution directly.  
  - Fast, simple, and widely used.  
  - Produces blocky, pixelated results.  
  - Useful for storage savings, ML preprocessing, or simulating low-res devices.  

- **SVD Truncation** reduces *detail* rather than pixel count.  
  - Keeps structure, loses fine detail.  
  - Produces smoother, blurred images.  
  - Useful for compression, denoising, and analysis.  

---

### When to use each?
- Use **downsampling** when you care about resolution, file size, or speed.  
- Use **SVD truncation** when you want to smooth/compress while keeping the main structure intact.  

Together, they show two complementary ways of "de-enhancement":  
one pixel-based, one linear-algebra-based.